## main.py

In [1]:

from fastapi import FastAPI
from pydantic import BaseModel
import networkx as nx
import pandas as pd

app = FastAPI(title="API de Grafos com Insights")

class CSVInput(BaseModel):
    ajuste: list
    cancelamento: list
    inventario: list
    devolucao: list

def construir_grafo(ajuste, cancelamento, inventario, devolucao):
    G = nx.DiGraph()

    ajuste_df = pd.DataFrame(ajuste)
    cancelamento_df = pd.DataFrame(cancelamento)
    inventario_df = pd.DataFrame(inventario)
    devolucao_df = pd.DataFrame(devolucao)

    # Ajustes
    for _, row in ajuste_df.iterrows():
        event_ajuste = f"AJUSTE_{row['id_loja']}_{row['id_sku']}_{row['data_ocorrencia']}"
        G.add_node(event_ajuste, tipo='Ajuste')
        G.add_edge(f"LOJA_{row['id_loja']}", event_ajuste)
        G.add_edge(f"SKU_{row['id_sku']}", event_ajuste)

    # Cancelamentos
    for _, row in cancelamento_df.iterrows():
        event_cancel = f"CANCELAMENTO_{row['id_loja']}_{row['id_sku']}_{row['data_ocorrencia']}"
        G.add_node(event_cancel, tipo='Cancelamento')
        G.add_edge(f"LOJA_{row['id_loja']}", event_cancel)
        G.add_edge(f"SKU_{row['id_sku']}", event_cancel)
        if pd.notna(row['id_usuario']):
            G.add_edge(f"USUARIO_{row['id_usuario']}", event_cancel)

    # Inventário
    for _, row in inventario_df.iterrows():
        event_inv = f"INVENTARIO_{row['id_loja']}_{row['id_sku']}_{row['data_ocorrencia']}"
        G.add_node(event_inv, tipo='Inventario')
        G.add_edge(f"LOJA_{row['id_loja']}", event_inv)
        G.add_edge(f"SKU_{row['id_sku']}", event_inv)

    # Devoluções
    for _, row in devolucao_df.iterrows():
        event_dev = f"DEVOLUCAO_{row['id_loja']}_{row['id_sku']}_{row['data_ocorrencia']}"
        G.add_node(event_dev, tipo='Devolucao')
        G.add_edge(f"LOJA_{row['id_loja']}", event_dev)
        G.add_edge(f"SKU_{row['id_sku']}", event_dev)

    return G

@app.post("/construir_grafo")
def construir(input_data: CSVInput):
    G = construir_grafo(
        input_data.ajuste,
        input_data.cancelamento,
        input_data.inventario,
        input_data.devolucao
    )

    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    degree_centrality = nx.degree_centrality(G)
    betweenness = nx.betweenness_centrality(G)
    components = [list(c) for c in nx.strongly_connected_components(G)]

    insights = {
        "numero_nos": n_nodes,
        "numero_arestas": n_edges,
        "grau_medio": sum(dict(G.degree()).values()) / n_nodes if n_nodes > 0 else 0,
        "centralidade_grau_top5": sorted(degree_centrality.items(), key=lambda x: -x[1])[:5],
        "betweenness_top5": sorted(betweenness.items(), key=lambda x: -x[1])[:5],
        "componentes": components
    }

    return {
        "grafo": nx.node_link_data(G),
        "insights": insights
    }


## requirements.txt

In [ ]:

%%writefile requirements.txt
fastapi
uvicorn
pandas
networkx


Writing requirements.txt


## Dockerfile

In [ ]:

%%writefile Dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY main.py .

EXPOSE 8000

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]


## Simulação de Request/Response

In [ ]:

import requests

url = "http://localhost:8000/construir_grafo"
payload = {
  "ajuste": [
    {"id_loja": 101, "id_sku": "ABC123", "data_ocorrencia": "2025-06-01",
     "qtd_antiga": 10, "quantidade": 12, "tipo_ajuste": "ENTRADA NO ESTOQUE"}
  ],
  "cancelamento": [
    {"id_loja": 101, "id_sku": "ABC123", "data_ocorrencia": "2025-06-02",
     "valor": 199.90, "idorcamento": 4567, "idcondicaopagamento": 3,
     "confirmado_cancelado": True, "ativo_cancelado": True, "id_usuario": 88}
  ],
  "inventario": [
    {"id_loja": 101, "id_sku": "ABC123", "data_ocorrencia": "2025-06-03",
     "qtd_inventario": 8}
  ],
  "devolucao": [
    {"id_loja": 101, "id_sku": "ABC123", "data_ocorrencia": "2025-06-04",
     "motivo": "TROCA DE NUMERAÇÃO"}
  ]
}

# Simulação de resposta esperada
simulated_response = {
  "grafo": {
    "directed": True,
    "multigraph": False,
    "nodes": [
      {"id": "AJUSTE_101_ABC123_2025-06-01", "tipo": "Ajuste"},
      {"id": "CANCELAMENTO_101_ABC123_2025-06-02", "tipo": "Cancelamento"},
      {"id": "INVENTARIO_101_ABC123_2025-06-03", "tipo": "Inventario"},
      {"id": "DEVOLUCAO_101_ABC123_2025-06-04", "tipo": "Devolucao"},
      {"id": "LOJA_101"}, {"id": "SKU_ABC123"}, {"id": "USUARIO_88"}
    ],
    "links": [
      {"source": "LOJA_101", "target": "AJUSTE_101_ABC123_2025-06-01"},
      {"source": "SKU_ABC123", "target": "AJUSTE_101_ABC123_2025-06-01"},
      {"source": "LOJA_101", "target": "CANCELAMENTO_101_ABC123_2025-06-02"},
      {"source": "SKU_ABC123", "target": "CANCELAMENTO_101_ABC123_2025-06-02"},
      {"source": "USUARIO_88", "target": "CANCELAMENTO_101_ABC123_2025-06-02"},
      {"source": "LOJA_101", "target": "INVENTARIO_101_ABC123_2025-06-03"},
      {"source": "SKU_ABC123", "target": "INVENTARIO_101_ABC123_2025-06-03"},
      {"source": "LOJA_101", "target": "DEVOLUCAO_101_ABC123_2025-06-04"},
      {"source": "SKU_ABC123", "target": "DEVOLUCAO_101_ABC123_2025-06-04"}
    ]
  },
  "insights": {
    "numero_nos": 7,
    "numero_arestas": 9,
    "grau_medio": 2.57,
    "centralidade_grau_top5": [
      ["LOJA_101", 0.33],
      ["SKU_ABC123", 0.33],
      ["CANCELAMENTO_101_ABC123_2025-06-02", 0.16],
      ["AJUSTE_101_ABC123_2025-06-01", 0.16],
      ["INVENTARIO_101_ABC123_2025-06-03", 0.16]
    ],
    "betweenness_top5": [
      ["SKU_ABC123", 0.25],
      ["LOJA_101", 0.20],
      ["CANCELAMENTO_101_ABC123_2025-06-02", 0.0],
      ["AJUSTE_101_ABC123_2025-06-01", 0.0],
      ["INVENTARIO_101_ABC123_2025-06-03", 0.0]
    ],
    "componentes": [
      ["LOJA_101","AJUSTE_101_ABC123_2025-06-01","CANCELAMENTO_101_ABC123_2025-06-02",
       "INVENTARIO_101_ABC123_2025-06-03","DEVOLUCAO_101_ABC123_2025-06-04","SKU_ABC123","USUARIO_88"]
    ]
  }
}

print(simulated_response)
